# Adding county info and finalizing dataset

In [3]:
import numpy as np
import numpy.random as npr
import pandas as pd

import json
import requests

import pickle

import time

import matplotlib
import matplotlib.pyplot as plt

import altair as alt
from vega_datasets import data

from scipy.spatial import KDTree

import plotly.express as px
import plotly.graph_objects as go

import glob
import os

In [4]:
disabilities = ['DOUT', 'DPHY', 'DREM', 'DDRS']

In [5]:
columns = ['Address',
 'Capacity',
 'City',
 'County',
 'Date Accessed',
 'Email Address',
 'Facility ID',
 'Facility Name',
 'Latitude',
 'License Number',
 'Licensee',
 'Longitude',
 'Ownership Type',
 'Phone Number',
 'State',
 'State Facility Type 1 Literal',
 'State Facility Type 2 Literal',
 'Zip Code']

cols_to_title = ['City', 'County', 'Facility Name']
cols_to_lower = ['Email Address']
cols_to_upper = ['Address', 'State Facility Type 1 Literal', 'State Facility Type 2 Literal', 'Ownership Type', 'Licensee']

## Functions

In [6]:
# cleaner function
# adds state abbreviation
# fixes capitalization
# removes some duplicates and shows you how many remaining
def cleaner(df):
    # finding wrong columns
    print('Extraneous columns are:\n{}'.format(list(set(df.columns) - set(columns))))

    # df['State'] = state

    # fixing capitalization
    for col in df.columns:
        if   col in cols_to_title: df[col] = df[col].str.title()
        elif col in cols_to_lower: df[col] = df[col].str.lower()
        elif col in cols_to_upper: df[col] = df[col].str.upper()


    if 'License Number' in df.columns:
        df.drop_duplicates(['Facility Name', 'License Number'], inplace=True)
        print('{} ALFs, {} unique License Numbers, {} NA License Numbers'.format(df.shape[0], df['License Number'].unique().shape[0], df['License Number'].isna().sum()))
    elif 'Facility ID' in df.columns:
        df.drop_duplicates(['Facility Name', 'Facility ID'], inplace=True)
        print('{} ALFs, {} unique Facility IDs, {} NA Facility IDs'.format(df.shape[0], df['Facility ID'].unique().shape[0], df['Facility ID'].isna().sum()))
    else:
        df.drop_duplicates(['Facility Name', 'Address'], inplace=True)
        print('No License Number or Facility ID')

### Functions to get disability data

In [7]:
# takes a row of df and returns 0 for no disability, 1 for ALF, 2 for NH
disability_status_helper = lambda x: 1 if x == 1 else 0

def get_disability_status(row):
    if row['DDRS'] == 1:
        return 2
    elif row[disabilities].apply(disability_status_helper).sum() >= 2:
        return 2
    elif row[disabilities].apply(disability_status_helper).sum() == 1:
        return 1
    else:
        return 0

In [8]:
# NEW DISABILITY METRIC
def get_disability_status_new(row):
    if row['DDRS'] == 1:
        return 1
    elif row[disabilities].apply(disability_status_helper).sum() >= 2:
        return 1
    else:
        return 0

In [9]:
# takes a df of disability data and returns column of disability status
def get_disability_status_column(df):
    temp = df.apply(get_disability_status, axis=1)
    return temp

In [10]:
# takes a df of disability data and returns column of disability status
def get_disability_status_column_new(df):
    temp = df.apply(get_disability_status_new, axis=1)
    return temp

In [11]:
# takes a df with a disability status column and populates the number of people in each puma with 'ALF' and 'NH' needs and returns dictionary
def count_disabilities(df):
    disability_dict = {'ALF':{}, 'NH':{}}
    for puma in df['PUMA'].unique():
        temp = df[df['PUMA'] == puma]

        disability_dict['ALF'][puma] = temp.loc[temp['Disability Status'] == 1, 'PWGTP'].sum()
        disability_dict['NH'][puma] = temp.loc[temp['Disability Status'] == 2, 'PWGTP'].sum()
    return disability_dict

In [12]:
# function that takes list of PUMA's and returns query string to get data from the PUMA's
def query_str(pumas):
    return url_base + '7950000US' + ',7950000US'.join(pumas)

In [13]:
# takes state abbreviation and returns df of relevent pums data for all PUMA's in state
# have to break request into chunks so census website gives proper response
# no request has more then 50 PUMA's in it
def get_pums_data(state):
    temp = puma_codes_df[puma_codes_df['State Code'] == state_code_dict[state]]['Full Code'].values

    num_parts = np.ceil(temp.shape[0] / 50)
    temp_split = np.array_split(temp, num_parts)

    urls = list(map(query_str, temp_split))
    jsons = [requests.get(url).json() for url in urls]
    dfs = [pd.DataFrame(json[1:], columns=json[0]).astype(int) for json in jsons]
    return pd.concat(dfs)

### Functions to calculate distances

In [14]:
# takes a row of county df with lat long and returns nearest facilities to satisfy ALF need * scalar
def get_alf_neighbors(row, scalar, capacity_type):
    assert scalar > 0 and scalar <= 1

    coord = [row['Latitude'], row['Longitude']]
    num_facilities = 5

    d, i = alf_kdtree.query(coord, k = num_facilities)

    if county_disability_dict.get(row['FIPS']) is None:
        print("Dictionary of county disability information doesn't have county with FIPS {}".format(row['FIPS']))
        return 0

    while alf_df.iloc[i][capacity_type].sum() < (county_disability_dict[row['FIPS']]['ALF'] * scalar):
        num_facilities += 5
        d, i = alf_kdtree.query(coord, k = num_facilities)
    return alf_df.iloc[i]

In [15]:
# takes a row of county df with lat long and returns nearest facilities to satisfy NH need * scalar
def get_nh_neighbors(row, scalar):
    assert scalar > 0 and scalar <= 1

    coord = [row['Latitude'], row['Longitude']]
    num_facilities = 5

    d, i = nh_kdtree.query(coord, k = num_facilities)

    if county_disability_dict.get(row['FIPS']) is None:
        print("Dictionary of county disability information doesn't have county with FIPS {}".format(row['FIPS']))
        return 0

    while nh_df.iloc[i]['Number of Certified Beds'].sum() < (county_disability_dict[row['FIPS']]['NH'] * scalar):
        num_facilities += 5
        d, i = nh_kdtree.query(coord, k = num_facilities)
    return nh_df.iloc[i]

In [16]:
# takes a row and a df and returns mean distance in miles from the row's lat/long to lat/long of each item in df, using the Haversine formula. See https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula for where the code came from
def mean_distance(row, df):
    p = np.pi/180
    return np.mean(3959.87433 * 2 * np.arcsin(np.sqrt(  0.5 -  np.cos((row['Latitude']-df['Latitude'])*p)/2 + np.cos(df['Latitude']*p) * np.cos(row['Latitude']*p) * (1-np.cos((row['Longitude'] - df['Longitude'])*p))/2)))

In [17]:
# combine neighbors and mean distance
def get_mean_distance_to_alf(row, scalar, capacity_type):
    return mean_distance(row, get_alf_neighbors(row, scalar, capacity_type))

# combine neighbors and mean distance
def get_mean_distance_to_nh(row, scalar):
    return mean_distance(row, get_nh_neighbors(row, scalar))

## Variables

In [19]:
# dictionary for disability data of each state and each PUMA
with open("../alf-datasets/national/disability-puma-data.pkl", 'rb') as f:
    puma_disability_dict = pickle.load(f)

with open("../alf-datasets/national/disability-puma-data-new.pkl", 'rb') as f:
    puma_disability_dict_new = pickle.load(f)

# ALF dataset
# alf_df = pd.read_pickle('alf-datasets/national/national-dataset-1-with-coordinates.pkl')
# alf_df.drop(alf_df[alf_df['State'] == '52'].index, inplace = True) # dropping one weird row
# grabbing new dataset as of 8/27/21 with all states but AK, ND, and VA
alf_df = pd.read_csv('../alf-datasets/national/national-dataset-51states-with-coords-and-county-10-28-21.csv')

# list of states that we have ALF data for
states_with_alf_data = alf_df['State'].unique()

# NH dataset
nh_df = pd.read_pickle('../../nh/data/nh-data/nh-long-1.pkl')

# Useful state info dataset we've already made
state_info_df = pd.read_pickle('../../nh/data/state-data/state-info-with-extras.pkl')

# dictionary of state abbriviation to state code using our state_info dataset
state_code_dict = dict(zip(state_info_df['abbreviation'], state_info_df['id']))

# PUMA county equivalence file and adding useful columns
puma_county_equivalence_df = pd.read_csv('../alf-datasets/national/puma-county-equivalence.csv')
puma_county_equivalence_df['Full Puma Code'] = puma_county_equivalence_df.apply(lambda x: str(x['State code']).zfill(2) + str(x['PUMA (2012)']).zfill(5), axis=1)

puma_county_equivalence_df['ALF need'] = puma_county_equivalence_df.apply(lambda x: puma_disability_dict[x['State abbreviation']]['ALF'][x['PUMA (2012)']], axis=1)
puma_county_equivalence_df['ALF need new'] = puma_county_equivalence_df.apply(lambda x: puma_disability_dict_new[x['State abbreviation']]['ALF'][x['PUMA (2012)']], axis=1)
puma_county_equivalence_df['NH need'] = puma_county_equivalence_df.apply(lambda x: puma_disability_dict[x['State abbreviation']]['NH'][x['PUMA (2012)']], axis=1)



puma_county_equivalence_df['Scaled county ALF need'] = puma_county_equivalence_df['puma12 to county allocation factor'] * puma_county_equivalence_df['ALF need']
puma_county_equivalence_df['Scaled county ALF need new'] = puma_county_equivalence_df['puma12 to county allocation factor'] * puma_county_equivalence_df['ALF need new']
puma_county_equivalence_df['Scaled county NH need'] = puma_county_equivalence_df['puma12 to county allocation factor'] * puma_county_equivalence_df['NH need']

# getting county fips and lat/long coord info
county_info_df = pd.read_csv('../alf-datasets/national/county-coords.csv')
county_info_df['Latitude'] = county_info_df['Latitude'].apply(lambda x: float(x[1:-1]))
county_info_df['Longitude'] = county_info_df['Longitude'].apply(lambda x: -1 * float(x[1:-1]))

# 65+ population per county (2015-2019)
county_oldpopulation_df = pd.read_csv('../alf-datasets/national/county-data/county-age-65.csv')
county_oldpopulation_df.columns = ['County', 'State', 'FIPS', 'Formatted FIPS', '65+ Population']

# PUMA codes
puma_codes_df = pd.read_csv('../alf-datasets/national/2010-puma-names.csv')
puma_codes_df.drop(columns='PUMA NAME', inplace=True)
puma_codes_df['Full Code'] = puma_codes_df.apply(lambda x: str(x['STATEFP']).zfill(2) + str(x['PUMA5CE']).zfill(5), axis=1)
puma_codes_df.rename(columns={'STATEFP':'State Code','PUMA5CE':'PUMA Code'}, inplace=True)

# just add puma codes to end of this url and use for request
url_base = 'https://api.census.gov/data/2019/acs/acs1/pums?get=PWGTP,DOUT,DREM,DPHY,DDRS&ucgid='

## STUFF

In [20]:
# converts disability data about PUMA's into disability data about counties
county_disability_dict = {}
for county in puma_county_equivalence_df['County code'].unique():
    county_disability_dict[county] = puma_county_equivalence_df[puma_county_equivalence_df['County code'] == county]['Scaled county ALF need new'].sum()
                                 
# APPARRENTLY 2270 IS A COUNTY WE ARE MISSING SO JUST GONNA SET IT TO 0
county_disability_dict[2270] = 0

In [21]:
alf_df['Total County AL Need'] = [int(county_disability_dict[fips]) if fips in county_disability_dict.keys() else np.nan for fips in alf_df['County FIPS'] ]

In [22]:
alf_df

,Facility ID,Facility Name,Address,City,State,Zip Code,Phone Number,County,Licensee,State Facility Type 2 Literal,State Facility Type 1 Literal,Date Accessed,License Number,Capacity,Email Address,Ownership Type,Latitude,Longitude,County FIPS,Total County AL Need
0,NaN,American House Keene,197 WATER ST,Keene,NH,3431,(603) 352-1282,Cheshire,NaN,RESIDENTIAL CARE FACILITY,ASSISTED LIVING RESIDENCE,8/3/21,4305,144.0,NaN,NaN,42.929853,-72.269919,33005.0,4613.0
1,NaN,Artaban House,40 PLOWSHARE LANE,Greenfield,NH,3047,(603) 547-3717,Hillsborough,NaN,RESIDENTIAL CARE FACILITY,ASSISTED LIVING RESIDENCE,8/3/21,3111,6.0,NaN,NaN,42.905206,-71.851984,33011.0,20336.0
2,NaN,Assisted Living At Pine Hill,35 NORTH LOWELL RD,Windham,NH,3087,(603) 506-6626,Rockingham,NaN,RESIDENTIAL CARE FACILITY,ASSISTED LIVING RESIDENCE,8/3/21,4480,16.0,NaN,NaN,42.815648,-71.301571,33015.0,15033.0
3,NaN,Austin Home,532 WHITE PLAINS ROAD,Webster,NH,3303,(603) 456-3525,Merrimack,NaN,RESIDENTIAL CARE FACILITY,ASSISTED LIVING RESIDENCE,8/3/21,2923,15.0,NaN,NaN,43.314164,-71.750532,33013.0,7198.0
4,NaN,Beaver Lake Lodge,38 NORTH SHORE ROAD,Derry,NH,3038,(603) 434-5683,Rockingham,NaN,RESIDENTIAL CARE FACILITY,ASSISTED LIVING RESIDENCE,8/3/21,4481,16.0,NaN,NaN,42.908671,-71.292571,33015.0,15033.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44633,33429,The Meadows Assisted Living Inc,2300 SE 28TH STREET,Bentonville,AR,72712,4792739969,Benton,NaN,NaN,ASSISTED LIVING FACILITY #2,8/27/21,NaN,81.0,anns@meadowsinbentonville.com,NaN,36.341100,-94.186337,5007.0,13121.0
44634,34035,The Arbors Assisted Living Facilities Llc Dba ...,697 BRADLEY COVE ROAD,Russellville,AR,72802,4792193902,Pope,NaN,NaN,ASSISTED LIVING FACILITY #2,8/27/21,NaN,75.0,csamuels@arborsofrussellville.com,NaN,35.284817,-93.076108,5115.0,5554.0
44635,34151,Crown Point Retirement Center Llc,510 SOUTH ROSE STREET,Sheridan,AR,72150,8709424623,Grant,NaN,NaN,ASSISTED LIVING FACILITY #2,8/27/21,NaN,69.0,alicia@crownpointretirement.com,NaN,34.302101,-92.405241,5053.0,1026.0
44636,35211,Crestview Senior Living,3310 WEST MAIN STREET,Cabot,AR,72023,5015298337,Lonoke,NaN,NaN,ASSISTED LIVING FACILITY #2,8/27/21,NaN,75.0,april.spencer@crestviewsl.com,NaN,34.983744,-92.060462,5085.0,4201.0


In [27]:
path = "../alf-datasets/national/county-data/for-table"

# gets county names, state, fips code, etc...
county_basic_info_df = pd.read_csv(glob.glob(os.path.join(path, '*'))[0]).iloc[:,:4]

# gets info for each dataset in county-data folder
county_dfs = pd.concat([pd.read_csv(loc).iloc[:,4] for loc in glob.glob(os.path.join(path, '*'))], axis=1)

county_df = pd.concat([county_basic_info_df, county_dfs], axis=1)

county_df['Formatted FIPS'] = county_df['FIPS Code'].astype(str).str.zfill(5)

In [28]:
county_df

,County,State,FIPS Code,Formatted FIPS,"Estimated percent of all people 65 or older, between 2015-2019.","Estimated median age of all people, between 2015-2019.","Estimated percent of all households that own a home, between 201","Estimated percent of people with at least a Bachelor's degree, b",Estimated percent of all people who were Black or African Americ,"Estimated median value of an owner-occupied home, between 2015-2","Estimated percent of all people who were Hispanic or Latino, bet","Estimated percent of all people 85 or older, between 2015-2019.","Estimated median income of a household, between 2015-2019.",Unemployment rate in 2020.,Estimated percent of people with less than a high school diploma,"Estimated percent of all people who were White, between 2015-201",Estimated ratio of men to women (number of males per hundred fem,Estimated percent of all people that are living in poverty as of
0,Autauga,AL,1001.0,1001.0,14.96,38.0,73.29,26.57,19.03,154500.0,2.83,1.60,58731.0,4.9,11.48,76.79,95.0,15.19
1,Baldwin,AL,1003.0,1003.0,19.98,43.0,75.25,31.86,9.26,197900.0,4.56,1.90,58320.0,5.6,9.19,86.21,95.0,10.35
2,Barbour,AL,1005.0,1005.0,18.57,40.0,60.90,11.58,47.58,90700.0,4.36,1.60,32525.0,7.0,26.79,46.80,112.0,30.67
3,Bibb,AL,1007.0,1007.0,15.93,41.0,74.42,10.38,22.29,92800.0,2.57,1.97,47542.0,6.6,20.94,76.79,117.0,18.13
4,Blount,AL,1009.0,1009.0,17.90,41.0,78.78,13.09,1.61,127800.0,9.26,1.83,49358.0,4.1,19.51,95.46,98.0,13.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3240,NaN,NaN,NaN,00nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3241,NaN,NaN,NaN,00nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3242,NaN,NaN,NaN,00nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3243,NaN,NaN,NaN,00nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
new_col_titles = ['County', 'State', 'FIPS Code', 'Formatted FIPS', 'County Percent of Population 65 or Older', 'County Median Age', 'County Homeownership Rate', 'County College Education or Higher Rate', 'County Percent Black Population', 'County Median Home Value of Owned Homes', 'County Percent Hispanic Population', 'County Percent of Population 85 or Older', 'County Median Household Income', 'County Unemployment Rate', 'County Less Than High School Diploma Rate', 'County Percent Whilte Population', 'County Gender Ratio', 'County Poverty Rate']

county_df.columns = new_col_titles

In [32]:
county_df

,County,State,FIPS Code,Formatted FIPS,County Percent of Population 65 or Older,County Median Age,County Homeownership Rate,County College Education or Higher Rate,County Percent Black Population,County Median Home Value of Owned Homes,County Percent Hispanic Population,County Percent of Population 85 or Older,County Median Household Income,County Unemployment Rate,County Less Than High School Diploma Rate,County Percent Whilte Population,County Gender Ratio,County Poverty Rate
0,Autauga,AL,1001.0,1001.0,14.96,38.0,73.29,26.57,19.03,154500.0,2.83,1.60,58731.0,4.9,11.48,76.79,95.0,15.19
1,Baldwin,AL,1003.0,1003.0,19.98,43.0,75.25,31.86,9.26,197900.0,4.56,1.90,58320.0,5.6,9.19,86.21,95.0,10.35
2,Barbour,AL,1005.0,1005.0,18.57,40.0,60.90,11.58,47.58,90700.0,4.36,1.60,32525.0,7.0,26.79,46.80,112.0,30.67
3,Bibb,AL,1007.0,1007.0,15.93,41.0,74.42,10.38,22.29,92800.0,2.57,1.97,47542.0,6.6,20.94,76.79,117.0,18.13
4,Blount,AL,1009.0,1009.0,17.90,41.0,78.78,13.09,1.61,127800.0,9.26,1.83,49358.0,4.1,19.51,95.46,98.0,13.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3240,NaN,NaN,NaN,00nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3241,NaN,NaN,NaN,00nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3242,NaN,NaN,NaN,00nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3243,NaN,NaN,NaN,00nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
new_col_titles[4:]

['County Percent of Population 65 or Older',
 'County Median Age',
 'County Homeownership Rate',
 'County College Education or Higher Rate',
 'County Percent Black Population',
 'County Median Home Value of Owned Homes',
 'County Percent Hispanic Population',
 'County Percent of Population 85 or Older',
 'County Median Household Income',
 'County Unemployment Rate',
 'County Less Than High School Diploma Rate',
 'County Percent Whilte Population',
 'County Gender Ratio',
 'County Poverty Rate']

In [34]:
for col in new_col_titles[4:]:
    temp_dict = dict(zip(county_df['FIPS Code'], county_df[col]))
    alf_df[col] = [temp_dict[fips] if fips in temp_dict.keys() else np.nan for fips in alf_df['County FIPS']]

In [35]:
alf_df.to_csv('../alf-datasets/national/national-dataset-51states-with-ACS-data-10-28-21.csv', index=False)

In [36]:
for col in alf_df.columns:
    print(col, (~alf_df[col].isna()).sum() / alf_df.shape[0], alf_df[col].isna().sum())

Facility ID 0.6430395627044222 15934
Facility Name 0.9999551951252297 2
Address 0.9956091222725032 196
City 0.9786280747345311 954
State 1.0 0
Zip Code 0.9688606120345894 1390
Phone Number 0.969532685156145 1360
County 0.9998431829383037 7
Licensee 0.6567946592589273 15320
State Facility Type 2 Literal 0.40427438505309377 26592
State Facility Type 1 Literal 0.9883507325597025 520
Date Accessed 0.9965948295174515 152
License Number 0.4680317218513374 23746
Capacity 0.8588646444733187 6300
Email Address 0.34743940140687307 29129
Ownership Type 0.2672162731305166 32710
Latitude 0.9998655853756889 6
Longitude 0.9998655853756889 6
County FIPS 0.9998431829383037 7
Total County AL Need 0.9998431829383037 7
County Percent of Population 65 or Older 0.9997759756261482 10
County Median Age 0.9997759756261482 10
County Homeownership Rate 0.9997759756261482 10
County College Education or Higher Rate 0.9997759756261482 10
County Percent Black Population 0.9997759756261482 10
County Median Home Value

In [83]:
(~alf_df['Address'].isna()).sum()

43634

In [91]:
alf_df['Capacity'].sum(), alf_df.shape[0]

(1202864.0, 43830)